In [1]:
import os
import sys
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchmetrics.classification import MultilabelAccuracy
from torchmetrics.classification import MultilabelAUROC
import pytorch_lightning as pl
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn.functional as F
sys.path.append('../..')
from multi_modal_heart.model.ecg_net_attention import ECGEncoder,ECGAttentionAE
from multi_modal_heart.model.ecg_net import ECGAE
from multi_modal_heart.model.ecg_net import BenchmarkClassifier
from multi_modal_heart.ECG.ecg_dataset import ECGDataset
from torchmetrics import Accuracy

class LitClassifier(pl.LightningModule):
    def __init__(self,encoder,input_dim,num_classes=2,lr=1e-3,freeze_encoder=False):
        super().__init__()
        self.lr =lr
        self.freeze_encoder = freeze_encoder
        self.num_classes = num_classes
        self.encoder = encoder
        if self.freeze_encoder:
            self.encoder.eval()
            for param in self.encoder.parameters():
                param.requires_grad = False
        self.accu_metric = Accuracy(task="multiclass",num_classes=num_classes)
        self.test_feature_list =[]
        self.test_eid_list = []
        self.preds_list = []
        self.y_list = []
        self.test_feature  = None
        self.attention_map_classwise = {}
        for i in range(num_classes):
            self.attention_map_classwise[i] = []
        #### add classifier if use benchmark classifier
  
        #### add classifier if use benchmark classifier
        self.downsteam_net = BenchmarkClassifier(input_size=input_dim,hidden_size=128,output_size=num_classes)
    def forward(self, x, mask=None):
        latent_code = self.encoder.get_features_after_pooling(x,mask)
        # print (latent_code.shape)
        self.latent_code  = latent_code
        return self.downsteam_net(latent_code)
    
    def training_step(self, batch, batch_idx):
        x = batch[0]
        y= batch[1]
        if self.freeze_encoder:
            self.encoder.eval()
            with torch.no_grad():
                latent_code = self.encoder.get_features_after_pooling(x,mask=None)
        else:
            latent_code = self.encoder.get_features_after_pooling(x,mask=None)
        self.latent_code  = latent_code
        y_hat = self.downsteam_net(latent_code)
        # print (y_hat.shape)
        loss = F.cross_entropy(y_hat, y)
        self.log("train_loss", loss,prog_bar=True)
        return loss
    def test_step(self, batch, batch_idx):
        x = batch[0]
        y= batch[1]
        eid_list = batch[2]
        logits = self(x,mask=None)
        loss = F.cross_entropy(logits, y)
        self.log("test_loss", loss)
        ## save the feature
        preds = torch.argmax(logits, dim=1)
        last_hidden_feature = self.downsteam_net.get_features(self.latent_code)
        eid_list = batch[2]
        preds = torch.argmax(logits, dim=1)
        ## save the attention map
        lead_attention,_ = self.encoder.get_attention()
        # print(lead_attention[i].shape)
        ## make the [np.array] to numpy array
        # time_attention_map = np.array(time_attention_map)
        for i, eid in enumerate(eid_list):
            if preds[i]==y[i]:
                print(f"correct prediction: {y[i]}, eid: {eid}")
                # print(time_attention_map[:,i].shape)
                if y[i]==0:
                    self.attention_map_classwise[0].append([eid,lead_attention[i]])
                else: 
                    self.attention_map_classwise[1].append([eid,lead_attention[i]])

        self.test_feature_list.append(last_hidden_feature)
        self.test_eid_list.append(eid_list)
        self.preds_list.append(preds)
        self.y_list.append(y)

        self.accu_metric.update(preds, y)
        return loss
    def on_test_epoch_end(self):
        self.log("test_acc", self.accu_metric.compute())
        ## save the feature
        test_feature = torch.cat(self.test_feature_list,dim=0)
        test_eid = torch.cat(self.test_eid_list,dim=0)
        preds = torch.cat(self.preds_list,dim=0)
        y = torch.cat(self.y_list,dim=0)

        ## save the feature to the disk
        self.test_feature = test_feature
        self.test_eid = test_eid
        self.preds = preds
        self.y = y

    def validation_step(self, batch, batch_idx):
        x = batch[0]
        y= batch[1]
        y_hat = self(x,mask=None)
        loss = F.cross_entropy(y_hat, y)
        self.log("val_loss", loss)
        return loss
    def clear_cache(self):
        self.test_feature_list =[]
        self.test_eid_list = []
        self.preds_list = []
        self.y_list = []
        self.test_feature  = None
        self.attention_map_classwise={}
        for i in range(self.num_classes):
            self.attention_map_classwise[i] = []
    def configure_optimizers(self):
        return torch.optim.AdamW(filter(lambda p: p.requires_grad, self.parameters()), lr=self.lr, weight_decay=1e-4)
    
def print_result(probs,super_classes_labels, topk=1):
    probs, label_indices = torch.topk(probs, topk)
    probs = probs.tolist()
    label_indices = label_indices.tolist()
    for prob, idx in zip(probs, label_indices):
        label = super_classes_labels[idx]
        print(f'{label} ({idx}):', round(prob, 4))
def calc_hamming_score(y_true, y_pred):
    return (
        (y_true & y_pred).sum(axis=1) / (y_true | y_pred).sum(axis=1)
    ).mean()    
# ecg_net = ECGAttentionAE(num_leads=12, time_steps=1024, z_dims=512, linear_out=512, downsample_factor=5, base_feature_dim=4,if_VAE=False,
#                          use_attention_pool=False,no_linear_in_E=True, apply_lead_mask=False, no_time_attention=False)
# classification_net = LitClassifier(encoder=ecg_net.encoder,input_dim=512,num_classes=5)
# # checkpoint_path  ="../../log_finetune/ECG_attention_512_raw_no_attention_pool_no_linear_abl_no_time_attention_ms_resnet/checkpoints/last-v3.ckpt"
# # checkpoint_path  ="../../log_finetune/ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet_ECG2Text/checkpoints/last-v5.ckpt"
# checkpoint_path = "../../log_finetune/ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet/checkpoints/last-v8.ckpt"
# print (torch.load(checkpoint_path)["state_dict"].keys())
# mm_checkpoint = torch.load(checkpoint_path)["state_dict"]
# encoder_params = {(".").join(key.split(".")[1:]):value for key, value in mm_checkpoint.items() if str(key).startswith("encoder")}
# classification_params = {(".").join(key.split(".")[1:]):value for key, value in mm_checkpoint.items() if str(key).startswith("downsteam_net")}
# classification_net.encoder.load_state_dict(encoder_params)
# classification_net.downsteam_net.load_state_dict(classification_params)



In [2]:
## load MI data from UKB
import numpy as np
from scipy.stats import zscore
time_steps =608
mi_data_path = "/home/engs2522/project/multi-modal-heart/multi_modal_heart/toolkits/ukb/non_imaging_information/MI/batched_ecg_median_wave.npy"
healthy_data_path = "/home/engs2522/project/multi-modal-heart/multi_modal_heart/toolkits/ukb/non_imaging_information/non_CVD/batched_ecg_median_wave_1045.npy"
eid_list_mi= np.load("/home/engs2522/project/multi-modal-heart/multi_modal_heart/toolkits/ukb/non_imaging_information/MI/median_eid_list.npy")
eid_list_healthy = np.load("/home/engs2522/project/multi-modal-heart/multi_modal_heart/toolkits/ukb/non_imaging_information/non_CVD/selected_1045_healthy_subjects_eid_list.npy")
hf_data = np.load(mi_data_path)
healthy_data = np.load(healthy_data_path)
print('disease data shape',hf_data.shape)
hf_data = zscore(hf_data,axis=-1)
healthy_data = zscore(healthy_data,axis=-1)
hf_data = np.nan_to_num(hf_data)
healthy_data = np.nan_to_num(healthy_data)


## pad the data to 608
pad_num = (time_steps-healthy_data.shape[-1])//2
hf_data = np.pad(hf_data,((0,0),(0,0),(pad_num,pad_num)),"constant",constant_values=0)
healthy_data = np.pad(healthy_data,((0,0),(0,0),(pad_num,pad_num)),"constant",constant_values=0)


labels = np.concatenate([np.ones(hf_data.shape[0]),np.zeros(healthy_data.shape[0])])
eid_full_list = np.concatenate([eid_list_mi,eid_list_healthy])
data = np.concatenate([hf_data,healthy_data],axis=0)
print (data.shape)
print (labels.shape)
print (len(eid_full_list))
## append the eid to the label
labels_eid = np.zeros((labels.shape[0],2))
labels_eid[:,0] = labels
labels_eid[:,1] = eid_full_list
print (labels_eid.shape)
# labels_eid.shape

## split the data into train validate and test, 40% for train, 10% for validate, 50% for test
from sklearn.model_selection import train_test_split
X_trainval, X_test, y_trainval_eid, y_test_eid = train_test_split(data, labels_eid, test_size=0.5, random_state=42)
X_train, X_val, y_train_eid, y_val_eid= train_test_split(X_trainval, y_trainval_eid, test_size=0.1, random_state=42)
y_train = y_train_eid[:,0]
y_val = y_val_eid[:,0]
y_test = y_test_eid[:,0]
print ('num of training data:{}, MI ratio:{}'.format(X_train.shape[0],y_train.sum()))

print ('num of validation data:{}, MI ratio:{}'.format(X_val.shape[0],y_val.sum()))

print ('num of test data:{}, MI ratio:{}'.format(X_test.shape[0],y_test.sum()))


disease data shape (1044, 12, 600)
(2089, 12, 608)
(2089,)
2089
(2089, 2)
num of training data:939, MI ratio:463.0
num of validation data:105, MI ratio:51.0
num of test data:1045, MI ratio:530.0


In [3]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import pytorch_lightning as pl

batch_size  = 32
tensor_x = torch.Tensor(X_train) # transform to torch tensor
tensor_y = torch.Tensor(y_train).long()
tensor_eid = torch.Tensor(y_train_eid[:,1]).long()
my_train_dataset = TensorDataset(tensor_x,tensor_y,tensor_eid) # create your datset
my_dataloader = DataLoader(my_train_dataset,batch_size=batch_size) # create your dataloader

## validation data
tensor_x = torch.Tensor(X_val) # transform to torch tensor
tensor_y = torch.Tensor(y_val).long()
tensor_eid = torch.Tensor(y_val_eid[:,1]).long()

my_val_dataset =TensorDataset(tensor_x,tensor_y,tensor_eid) # create your datset
my_val_dataloader = DataLoader(my_val_dataset,batch_size=batch_size) # create your dataloader

## test data
tensor_x = torch.Tensor(X_test) # transform to torch tensor
tensor_y = torch.Tensor(y_test).long()
tensor_eid = torch.Tensor(y_test_eid[:,1]).long()
my_test_dataset = TensorDataset(tensor_x,tensor_y,tensor_eid) # create your datset
my_test_dataloader = DataLoader(my_test_dataset,batch_size=batch_size) # create your dataloader

In [4]:
import torch
import sys
sys.path.append('../../')
from multi_modal_heart.model.ecg_net import ECGAE
import torch.nn as nn
use_median_wave = True
time_steps = 608
pl.seed_everything(42)
model_name = "ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet_ECG2Text"

if model_name=="resnet1d101_512+benchmark_classifier_ms_resnet_finetune":
    ecg_net= ECGAE(encoder_type="resnet1d101",in_channels=12,ECG_length=time_steps,decoder_type="ms_resnet",
                    embedding_dim=256,latent_code_dim=512,
                    add_time=False,
                    encoder_mha = False,
                    apply_method="",
                    decoder_outdim=12)
    checkpoint_path = "/home/engs2522/project/multi-modal-heart/log_median/resnet1d101_512+benchmark_classifier_ms_resnet/checkpoints/checkpoint_best_loss.ckpt"
elif model_name=="resnet1d101_512+benchmark_classifier_ms_resnet_finetune_ECG2Text":
    ecg_net= ECGAE(encoder_type="resnet1d101",in_channels=12,ECG_length=time_steps,decoder_type="ms_resnet",
                    embedding_dim=256,latent_code_dim=512,
                    add_time=False,
                    encoder_mha = False,
                    apply_method="",
                    decoder_outdim=12)
    checkpoint_path = "/home/engs2522/project/multi-modal-heart/log_median/resnet1d101_512+benchmark_classifier_ms_resnet_ECG2Text/checkpoints/checkpoint_best_loss.ckpt"

elif model_name=="resnet1d101_512+benchmark_classifier_ms_resnet":
    ecg_net= ECGAE(encoder_type="resnet1d101",in_channels=12,ECG_length=time_steps,decoder_type="ms_resnet",
                    embedding_dim=256,latent_code_dim=512,
                    add_time=False,
                    encoder_mha = False,
                    apply_method="",
                    decoder_outdim=12)
    checkpoint_path = "/home/engs2522/project/multi-modal-heart/log_median_finetune/resnet1d101_512+benchmark_classifier_ms_resnet/checkpoints/checkpoint_best_loss.ckpt"
elif model_name=="resnet1d101_512+benchmark_classifier_ms_resnet_ECG2Text":
    ecg_net= ECGAE(encoder_type="resnet1d101",in_channels=12,ECG_length=time_steps,decoder_type="ms_resnet",
                    embedding_dim=256,latent_code_dim=512,
                    add_time=False,
                    encoder_mha = False,
                    apply_method="",
                    decoder_outdim=12)
    checkpoint_path = "/home/engs2522/project/multi-modal-heart/log_median_finetune/resnet1d101_512+benchmark_classifier_ms_resnet_ECG2Text/checkpoints/checkpoint_best_loss.ckpt"

elif model_name=="ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet":
    ecg_net  = ECGAttentionAE(num_leads=12, time_steps=time_steps, z_dims=512, linear_out=512, downsample_factor=5, base_feature_dim=4,if_VAE=False,use_attention_pool=False,
                         no_linear_in_E=True, apply_lead_mask=False)
    checkpoint_path = "../../log_median_finetune/ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet/checkpoints/checkpoint_best_loss.ckpt"
elif model_name=="ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet_ECG2Text":
    ecg_net  = ECGAttentionAE(num_leads=12, time_steps=time_steps, z_dims=512, linear_out=512, downsample_factor=5, base_feature_dim=4,if_VAE=False,use_attention_pool=False,
                         no_linear_in_E=True, apply_lead_mask=False)
    checkpoint_path = "../../log_median_finetune/ECG_attention_512_raw_no_attention_pool_no_linear_ms_resnet_ECG2Text/checkpoints/checkpoint_best_loss.ckpt"

elif model_name=="ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet":
    ecg_net  = ECGAttentionAE(num_leads=12, time_steps=time_steps, z_dims=512, linear_out=512, 
                            downsample_factor=5, base_feature_dim=4,if_VAE=False,
                            use_attention_pool=False,
                         no_linear_in_E=True, apply_lead_mask=False, no_lead_attention=False,no_time_attention=False)
    checkpoint_path = "../../log_median/ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet/checkpoints/checkpoint_best_loss.ckpt"
elif model_name=="ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet_ECG2Text":
    ecg_net  = ECGAttentionAE(num_leads=12, time_steps=time_steps, z_dims=512, linear_out=512, 
                            downsample_factor=5, base_feature_dim=4,if_VAE=False,
                            use_attention_pool=False,
                         no_linear_in_E=True, apply_lead_mask=False, no_lead_attention=False,no_time_attention=False)
    checkpoint_path = "../../log_median/ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet_ECG2Text/checkpoints/checkpoint_best_loss.ckpt"

else:
    raise NotImplementedError


num_classes = 2 ## for binary classification
freeze_encoder = True
train_from_scratch = False
classification_net = LitClassifier(encoder=ecg_net.encoder,input_dim=512,num_classes=num_classes,lr=1e-3,freeze_encoder=freeze_encoder)
# resnet_checkpoint = '../../log_finetune/resnet1d101_512+benchmark_classifier_ms_resnet/checkpoints/epoch=23-val_auroc:benchmark_classifier/val_macro_auc=0.91.ckpt'
checkpoint = torch.load(checkpoint_path)["state_dict"]
if not train_from_scratch:
    try:
        encoder_params = {(".").join(key.split(".")[2:]):value for key, value in checkpoint.items() if str(key).startswith("network.encoder")}
        classification_net.encoder.load_state_dict(encoder_params)
    except:
        encoder_params = {(".").join(key.split(".")[1:]):value for key, value in checkpoint.items() if str(key).startswith("encoder")}
        classification_net.encoder.load_state_dict(encoder_params)
    max_epochs = 50
else:
    max_epochs = 400

Global seed set to 42


no linear layer


In [5]:
import torch
import sys
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import os
tb_logger = TensorBoardLogger( f"./finetune_on_MI", name=model_name, version="")  
checkpoint_dir  = os.path.join(tb_logger.log_dir,"checkpoints")

checkpoint_callback_best_loss_min = pl.callbacks.ModelCheckpoint(dirpath=checkpoint_dir, 
                                                    filename='checkpoint_best_loss',
                                                    save_top_k=1, monitor="val_loss"
                                                    , mode='min',save_last=True)


callbacks=[
    # FineTuneLearningRateFinder(milestones=[5, 10],min_lr=1e-5, max_lr=1e-3, 
    #                             mode='linear', early_stop_threshold=4.0),
    checkpoint_callback_best_loss_min
    ]


trainer = pl.Trainer(accelerator="gpu",
                    devices=1, max_epochs=max_epochs,
                    logger=tb_logger,log_every_n_steps=1,check_val_every_n_epoch = 1,
                    callbacks=callbacks,
                    )
trainer.fit(classification_net, train_dataloaders=my_dataloader,val_dataloaders=my_val_dataloader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/engs2522/local/conda/envs/pytorch3d/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /home/engs2522/project/multi-modal-heart/multi_modal_heart/tasks/finetune_on_MI/ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet_ECG2Text/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name          | Type                | Params
------------------------------------------------------
0 | encoder       | ECGEncoder          | 1.3 M 
1 | accu_metric   | MulticlassAccuracy  | 0     
2 | downsteam_net | BenchmarkClassifier | 67.2 K
------------------------------------------------------
67.2 K    Trainable params
1.3 M     Non-trainable params
1.3 M     Tot

Sanity Checking: 0it [00:00, ?it/s]

/home/engs2522/local/conda/envs/pytorch3d/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 28 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/engs2522/local/conda/envs/pytorch3d/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 28 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


In [13]:
classification_net.clear_cache()
classification_net.eval()
classification_net.freeze()
result = trainer.test(classification_net,my_test_dataloader,ckpt_path="best")



Restoring states from the checkpoint path at /home/engs2522/project/multi-modal-heart/multi_modal_heart/tasks/finetune_on_MI/ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet_ECG2Text/checkpoints/checkpoint_best_loss-v13.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at /home/engs2522/project/multi-modal-heart/multi_modal_heart/tasks/finetune_on_MI/ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet_ECG2Text/checkpoints/checkpoint_best_loss-v13.ckpt


Testing: 0it [00:00, ?it/s]

correct prediction: 1, eid: 3546637
correct prediction: 0, eid: 5016936
correct prediction: 1, eid: 3781638
correct prediction: 0, eid: 2429902
correct prediction: 0, eid: 4264623
correct prediction: 1, eid: 2222340
correct prediction: 0, eid: 3099752
correct prediction: 1, eid: 4859890
correct prediction: 1, eid: 1279372
correct prediction: 1, eid: 4558831
correct prediction: 1, eid: 1485401
correct prediction: 0, eid: 2925804
correct prediction: 0, eid: 2353081
correct prediction: 0, eid: 2894893
correct prediction: 0, eid: 2972422
correct prediction: 1, eid: 2948213
correct prediction: 1, eid: 2493242
correct prediction: 0, eid: 3884530
correct prediction: 0, eid: 5254226
correct prediction: 0, eid: 2837935
correct prediction: 0, eid: 2824728
correct prediction: 1, eid: 3562794
correct prediction: 1, eid: 1161573
correct prediction: 1, eid: 3370850
correct prediction: 0, eid: 2732039
correct prediction: 0, eid: 4790341
correct prediction: 0, eid: 3058930
correct prediction: 1, eid: 

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7434588670730591     │
│         test_loss         │    0.6101080179214478     │
└───────────────────────────┴───────────────────────────┘

In [14]:
## visualize attention maps
## get attention map
attention_map_classwise = classification_net.attention_map_classwise
## norm group
print (len(attention_map_classwise))
norm_group_attention = attention_map_classwise[0]
hf_group_attention = attention_map_classwise[1]
print (len(norm_group_attention))
print (len(hf_group_attention))
norm_attention_maps= [v[1] for v in norm_group_attention]
hf_attention_maps = [v[1] for v in hf_group_attention]

norm_attention_maps = np.stack(norm_attention_maps,axis=0)
hf_attention_maps = np.stack(hf_attention_maps,axis=0)

average_attention_map_norm = np.mean(norm_attention_maps,axis=0)
average_attention_map_hf = np.mean(hf_attention_maps,axis=0)
## std variance
var_attention_map_norm = np.std(norm_attention_maps,axis=0)
var_attention_map_hf = np.std(hf_attention_maps,axis=0)

print(average_attention_map_hf.shape)

2
372
351
(12, 12)


In [8]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=1.5)
x_ticks = ["I","II","III","aVR","aVL","aVF","V1","V2","V3","V4","V5","V6"]
# cmap  =sns.palplot(sns.color_palette("Blues",12))
colormap = sns.color_palette("Greens")
fig,axes = plt.subplots(ncols=2,nrows=2,figsize=(20,20),dpi=500)

sns.heatmap(average_attention_map_norm,cmap=colormap,fmt=".2f",annot=False,xticklabels=x_ticks,yticklabels=x_ticks,ax=axes[0,0])
sns.heatmap(average_attention_map_hf,cmap=colormap,fmt=".2f",annot=False,xticklabels=x_ticks,yticklabels=x_ticks,ax=axes[0,1])

sns.heatmap(var_attention_map_norm,cmap=colormap,fmt=".2f",annot=False,xticklabels=x_ticks,yticklabels=x_ticks,ax=axes[1,0])
sns.heatmap(var_attention_map_hf,cmap=colormap,fmt=".2f",annot=False,xticklabels=x_ticks,yticklabels=x_ticks,ax=axes[1,1])

axes[0,0].set_title("control group [AVG]")
axes[0,1].set_title("MI group [AVG]")
axes[1,0].set_title("control group [STD]")
axes[1,1].set_title("MI group [STD]")

## change xticks
# plt.xticks(np.arange(0,12),x_ticks,rotation=45)
# plt.xlabel("leads")
# ## change yticks
# plt.yticks(np.arange(0,12),x_ticks,rotation=45)
## put x ticks on the top
# ax = plt.gca()
# ax.xaxis.tick_top()

Text(0.5, 1.0, 'MI group [STD]')

In [9]:
classification_net.test_feature.shape
## save the feature to the disk


torch.Size([1045, 128])

In [10]:
import numpy as np
feature_path = os.path.join(tb_logger.log_dir,"feature")
if not os.path.exists(feature_path):
    os.makedirs(feature_path)

result = trainer.test(classification_net,my_dataloader,ckpt_path="best")
result = trainer.test(classification_net,my_val_dataloader,ckpt_path="best")




Restoring states from the checkpoint path at /home/engs2522/project/multi-modal-heart/multi_modal_heart/tasks/finetune_on_MI/ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet_ECG2Text/checkpoints/checkpoint_best_loss-v13.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at /home/engs2522/project/multi-modal-heart/multi_modal_heart/tasks/finetune_on_MI/ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet_ECG2Text/checkpoints/checkpoint_best_loss-v13.ckpt


Testing: 0it [00:00, ?it/s]

correct prediction: 1, eid: 3928020
correct prediction: 1, eid: 2510196
correct prediction: 0, eid: 2705541
correct prediction: 0, eid: 4041082
correct prediction: 1, eid: 3654097
correct prediction: 0, eid: 4781151
correct prediction: 1, eid: 1461190
correct prediction: 1, eid: 2664427
correct prediction: 1, eid: 1211080
correct prediction: 1, eid: 3173053
correct prediction: 1, eid: 4014687
correct prediction: 1, eid: 1843302
correct prediction: 0, eid: 4706761
correct prediction: 0, eid: 1001657
correct prediction: 1, eid: 4139424
correct prediction: 1, eid: 5880752
correct prediction: 0, eid: 4607620
correct prediction: 1, eid: 2340672
correct prediction: 1, eid: 2587805
correct prediction: 0, eid: 2155795
correct prediction: 0, eid: 5502811
correct prediction: 1, eid: 1399552
correct prediction: 0, eid: 1516911
correct prediction: 1, eid: 4351013
correct prediction: 0, eid: 5020535
correct prediction: 1, eid: 3844696
correct prediction: 1, eid: 4777421
correct prediction: 1, eid: 

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7711693644523621     │
│         test_loss         │    0.3536125719547272     │
└───────────────────────────┴───────────────────────────┘

Restoring states from the checkpoint path at /home/engs2522/project/multi-modal-heart/multi_modal_heart/tasks/finetune_on_MI/ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet_ECG2Text/checkpoints/checkpoint_best_loss-v13.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from the checkpoint at /home/engs2522/project/multi-modal-heart/multi_modal_heart/tasks/finetune_on_MI/ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet_ECG2Text/checkpoints/checkpoint_best_loss-v13.ckpt


Testing: 0it [00:00, ?it/s]

correct prediction: 0, eid: 1937003
correct prediction: 1, eid: 2789791
correct prediction: 1, eid: 4505191
correct prediction: 0, eid: 1149392
correct prediction: 1, eid: 5681859
correct prediction: 1, eid: 1504452
correct prediction: 1, eid: 4918573
correct prediction: 0, eid: 3673011
correct prediction: 1, eid: 3578642
correct prediction: 1, eid: 1764852
correct prediction: 0, eid: 1131221
correct prediction: 1, eid: 5558315
correct prediction: 1, eid: 1028018
correct prediction: 1, eid: 5114984
correct prediction: 0, eid: 4068700
correct prediction: 1, eid: 2628002
correct prediction: 0, eid: 2611037
correct prediction: 0, eid: 1323215
correct prediction: 0, eid: 1368961
correct prediction: 1, eid: 5453470
correct prediction: 0, eid: 4580924
correct prediction: 0, eid: 3028124
correct prediction: 0, eid: 3286559
correct prediction: 0, eid: 2733297
correct prediction: 0, eid: 1816432
correct prediction: 1, eid: 2505859
correct prediction: 1, eid: 2679089
correct prediction: 0, eid: 

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7692676186561584     │
│         test_loss         │    0.6133000254631042     │
└───────────────────────────┴───────────────────────────┘

In [11]:
## do the same to the train and validation data
print (classification_net.test_feature.shape)

np.save(f"{feature_path}/test_last_hidden_feature.npy",classification_net.test_feature.cpu().numpy())
np.save(f"{feature_path}/test_eid.npy",classification_net.test_eid.cpu().numpy())
np.save(f"{feature_path}/test_preds.npy",classification_net.preds.cpu().numpy())
np.save(f"{feature_path}/test_y.npy",classification_net.y.cpu().numpy())
print('save to {}'.format(feature_path))

torch.Size([2089, 128])
save to ./finetune_on_MI/ECG_attention_512_finetuned_no_attention_pool_no_linear_ms_resnet_ECG2Text/feature
